In [1]:
import tensorflow as tf
from model_v2 import LanguageModel
import json
import numpy as np
from utils import batchify, get_batch
import time

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # pylint: disable=no-member
sess = tf.Session(config=config)

In [3]:
with open('102/checkpoints/model_configs.json', 'r') as inp:
    params = json.load(inp)

In [4]:
model = LanguageModel(**params, is_training=False)

In [5]:
model.build_model()
saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection('LSTM_SAVED_STATE')])
sess.run(tf.global_variables_initializer())

In [6]:
saver.restore(sess, '102/checkpoints/test/model.cpkt-278730')

INFO:tensorflow:Restoring parameters from 102/checkpoints/test/model.cpkt-278730


In [7]:
with open('wiki_bak/valid_char.npy', 'rb') as inp:
    test_char = np.load(inp)
with open('wiki_bak/valid_word.npy', 'rb') as inp:
    test_word = np.load(inp)
print(len(test_word), len(test_char))

11875464 11875464


In [8]:
test_word = batchify(test_word, 54).T
test_char = batchify(test_char, 54).T

In [9]:
fw_iy = tf.placeholder(dtype=tf.int32, shape=[None, None], name='fw_y')
bw_iy = tf.placeholder(dtype=tf.int32, shape=[None, None], name='bw_y')
test_loss = 0.5 * tf.add(
    tf.contrib.seq2seq.sequence_loss(
        logits=model.fw_model['decoder'],
        targets=fw_iy,
        weights=model.seq_masks,
        average_across_timesteps=True,
        average_across_batch=True),
    tf.contrib.seq2seq.sequence_loss(
        logits=model.bw_model['decoder'],
        targets=bw_iy,
        weights=model.seq_masks,
        average_across_timesteps=True,
        average_across_batch=True),
    name='test_loss'
)

In [10]:
def evaluate_step(test_word, test_char, bptt):
    start_time = time.time()
    total_loss = 0
    step = None
    for i in range(0, len(test_word), bptt):
        (fw_x, fw_y), (bw_x, bw_y) = get_batch(test_word, test_char, bptt, i, evaluate=True)
        loss = sess.run(
            test_loss,
            feed_dict={
                model.fw_inputs: fw_x,
                model.bw_inputs: bw_x,
                fw_iy: fw_y,
                bw_iy: bw_y,
                model.seq_lens: [fw_y.shape[0]] * fw_y.shape[1],
                model.reset_state: i == 0
            }
        )
        total_loss += loss * len(fw_y)
        print("Evaluate loss {}, time {}".format(loss, time.time()-start_time))
    total_loss /= len(test_word)
    print("Evaluate total loss {}, time {}".format(total_loss, time.time()-start_time))

In [11]:
evaluate_step(test_word, test_char, 50)

Evaluate loss 3.4096603393554688, time 0.881929874420166
Evaluate loss 3.237795829772949, time 1.2978971004486084
Evaluate loss 3.1060080528259277, time 1.719139575958252
Evaluate loss 3.0359838008880615, time 2.191359281539917
Evaluate loss 3.0510878562927246, time 2.662554979324341
Evaluate loss 3.1063549518585205, time 3.08052134513855
Evaluate loss 3.0391385555267334, time 3.554762363433838
Evaluate loss 3.1381137371063232, time 3.9705214500427246
Evaluate loss 3.082258701324463, time 4.3921637535095215
Evaluate loss 3.0440878868103027, time 4.808645009994507
Evaluate loss 3.0584795475006104, time 5.22758150100708
Evaluate loss 3.1349196434020996, time 5.644357204437256
Evaluate loss 3.174009084701538, time 6.063986778259277
Evaluate loss 3.1815226078033447, time 6.480318546295166
Evaluate loss 3.194918155670166, time 6.903514862060547
Evaluate loss 3.2353649139404297, time 7.3722243309021
Evaluate loss 3.232698678970337, time 7.7897584438323975
Evaluate loss 3.318013906478882, tim

Evaluate loss 3.1753153800964355, time 61.74192237854004
Evaluate loss 3.193125009536743, time 62.15927219390869
Evaluate loss 3.197653293609619, time 62.582813024520874
Evaluate loss 3.1093995571136475, time 63.00183606147766
Evaluate loss 3.139808416366577, time 63.419981479644775
Evaluate loss 3.0277793407440186, time 63.836408853530884
Evaluate loss 2.963683605194092, time 64.25656962394714
Evaluate loss 3.0521481037139893, time 64.67581582069397
Evaluate loss 3.0249269008636475, time 65.09437823295593
Evaluate loss 3.016221761703491, time 65.51178216934204
Evaluate loss 3.1690971851348877, time 65.929847240448
Evaluate loss 3.0794360637664795, time 66.34806370735168
Evaluate loss 3.2365477085113525, time 66.7714614868164
Evaluate loss 3.1778016090393066, time 67.19415378570557
Evaluate loss 3.2324037551879883, time 67.61082530021667
Evaluate loss 3.2210304737091064, time 68.02876257896423
Evaluate loss 3.177156448364258, time 68.44781827926636
Evaluate loss 3.274117946624756, time

Evaluate loss 3.2235801219940186, time 122.90970659255981
Evaluate loss 3.219487190246582, time 123.3374936580658
Evaluate loss 3.2223198413848877, time 123.75631642341614
Evaluate loss 3.1456799507141113, time 124.18389987945557
Evaluate loss 3.1058309078216553, time 124.6076271533966
Evaluate loss 3.103325843811035, time 125.0353090763092
Evaluate loss 3.1150925159454346, time 125.46062684059143
Evaluate loss 3.193009376525879, time 125.88479328155518
Evaluate loss 3.247452974319458, time 126.3046133518219
Evaluate loss 3.1899890899658203, time 126.73082971572876
Evaluate loss 3.1471657752990723, time 127.15545225143433
Evaluate loss 3.0818474292755127, time 127.58677530288696
Evaluate loss 3.2063961029052734, time 128.00509452819824
Evaluate loss 3.230678081512451, time 128.42717218399048
Evaluate loss 3.2336392402648926, time 128.8478491306305
Evaluate loss 3.1984686851501465, time 129.26869440078735
Evaluate loss 3.1492722034454346, time 129.69111585617065
Evaluate loss 3.07551860

Evaluate loss 3.2018635272979736, time 183.36809968948364
Evaluate loss 3.1600513458251953, time 183.79270911216736
Evaluate loss 3.1542470455169678, time 184.21272993087769
Evaluate loss 3.3323922157287598, time 184.63787055015564
Evaluate loss 3.324741840362549, time 185.05793523788452
Evaluate loss 3.20804762840271, time 185.48056292533875
Evaluate loss 3.2598516941070557, time 185.90362095832825
Evaluate loss 3.1319408416748047, time 186.32639694213867
Evaluate loss 3.0839884281158447, time 186.74915552139282
Evaluate loss 3.1715328693389893, time 187.16907405853271
Evaluate loss 3.117311477661133, time 187.59248542785645
Evaluate loss 3.1267166137695312, time 188.01220417022705
Evaluate loss 3.0903899669647217, time 188.43234848976135
Evaluate loss 3.2109992504119873, time 188.8558669090271
Evaluate loss 3.2571897506713867, time 189.27629685401917
Evaluate loss 3.1607472896575928, time 189.697829246521
Evaluate loss 3.1889548301696777, time 190.11415815353394
Evaluate loss 3.20433

Evaluate loss 3.1602814197540283, time 244.24095630645752
Evaluate loss 3.051543951034546, time 244.66081929206848
Evaluate loss 3.1063528060913086, time 245.07999897003174
Evaluate loss 3.230079174041748, time 245.50442838668823
Evaluate loss 3.090963840484619, time 245.93015599250793
Evaluate loss 3.162442445755005, time 246.35138535499573
Evaluate loss 3.133570671081543, time 246.77565503120422
Evaluate loss 3.023256301879883, time 247.19980001449585
Evaluate loss 3.184964656829834, time 247.62433052062988
Evaluate loss 3.2097532749176025, time 248.05015683174133
Evaluate loss 3.1915550231933594, time 248.46849465370178
Evaluate loss 3.07218861579895, time 248.892582654953
Evaluate loss 3.015530586242676, time 249.31581377983093
Evaluate loss 3.0202174186706543, time 249.7417459487915
Evaluate loss 3.084505319595337, time 250.16349244117737
Evaluate loss 3.0263354778289795, time 250.58664059638977
Evaluate loss 3.0800979137420654, time 251.0117597579956
Evaluate loss 3.1650133132934

...

Evaluate loss 3.1524760723114014, time 1592.0238132476807
Evaluate loss 3.1851699352264404, time 1592.4449481964111
Evaluate loss 3.1248929500579834, time 1592.8664770126343
Evaluate loss 3.1569292545318604, time 1593.288098335266
Evaluate loss 2.964531660079956, time 1593.7080390453339
Evaluate loss 2.9739596843719482, time 1594.1298785209656
Evaluate loss 2.8423514366149902, time 1594.5602533817291
Evaluate loss 2.9552035331726074, time 1594.9861121177673
Evaluate loss 2.9451334476470947, time 1595.4124541282654
Evaluate loss 2.878680467605591, time 1595.8344423770905
Evaluate loss 2.8621175289154053, time 1596.262696504593
Evaluate loss 2.974470376968384, time 1596.6895971298218
Evaluate loss 3.026606321334839, time 1597.108958721161
Evaluate loss 2.998241424560547, time 1597.534427165985
Evaluate loss 3.060882568359375, time 1597.9559442996979
Evaluate loss 3.2168478965759277, time 1598.3853273391724
Evaluate loss 3.1991138458251953, time 1598.8097739219666
Evaluate loss 3.09240198

Evaluate loss 3.1680989265441895, time 1653.0151345729828
Evaluate loss 3.145357370376587, time 1653.4415836334229
Evaluate loss 3.191779851913452, time 1653.8667821884155
Evaluate loss 3.172531843185425, time 1654.2895021438599
Evaluate loss 3.083834648132324, time 1654.709347486496
Evaluate loss 3.1431047916412354, time 1655.1348843574524
Evaluate loss 3.1379382610321045, time 1655.5556554794312
Evaluate loss 3.2264583110809326, time 1655.977471113205
Evaluate loss 3.2436962127685547, time 1656.401612520218
Evaluate loss 3.181378126144409, time 1656.8215372562408
Evaluate loss 3.124190092086792, time 1657.2475419044495
Evaluate loss 3.1470413208007812, time 1657.6703119277954
Evaluate loss 3.1667633056640625, time 1658.0956118106842
Evaluate loss 3.1113924980163574, time 1658.5175168514252
Evaluate loss 3.132544755935669, time 1658.9396831989288
Evaluate loss 3.1563663482666016, time 1659.3606226444244
Evaluate loss 3.1185858249664307, time 1659.7834358215332
Evaluate loss 3.09839653

Evaluate loss 3.0937366485595703, time 1713.6039836406708
Evaluate loss 3.0331993103027344, time 1714.0274019241333
Evaluate loss 3.071438789367676, time 1714.447728395462
Evaluate loss 3.1466562747955322, time 1714.8738667964935
Evaluate loss 3.1639318466186523, time 1715.2975521087646
Evaluate loss 3.1857399940490723, time 1715.720918416977
Evaluate loss 3.0054683685302734, time 1716.142616033554
Evaluate loss 3.0095396041870117, time 1716.5682301521301
Evaluate loss 3.0460712909698486, time 1716.9938247203827
Evaluate loss 3.145695924758911, time 1717.4153199195862
Evaluate loss 3.232628107070923, time 1717.833550453186
Evaluate loss 3.285337209701538, time 1718.2545533180237
Evaluate loss 3.216421127319336, time 1718.673049211502
Evaluate loss 3.176009178161621, time 1719.0932416915894
Evaluate loss 3.1769752502441406, time 1719.5167171955109
Evaluate loss 3.1169116497039795, time 1719.9384212493896
Evaluate loss 3.1606531143188477, time 1720.3587999343872
Evaluate loss 3.148479461

Evaluate loss 3.037386417388916, time 1774.540426492691
Evaluate loss 3.1236770153045654, time 1774.9672682285309
Evaluate loss 3.1264150142669678, time 1775.3904333114624
Evaluate loss 3.1055946350097656, time 1775.812817811966
Evaluate loss 3.217104196548462, time 1776.2316341400146
Evaluate loss 3.269193410873413, time 1776.6554951667786
Evaluate loss 3.2566919326782227, time 1777.079291343689
Evaluate loss 3.152801275253296, time 1777.5022468566895
Evaluate loss 3.207775592803955, time 1777.9259042739868
Evaluate loss 3.3869082927703857, time 1778.34636759758
Evaluate loss 3.301734685897827, time 1778.769817829132
Evaluate loss 3.1640942096710205, time 1779.1946501731873
Evaluate loss 3.162684440612793, time 1779.618158340454
Evaluate loss 3.2107291221618652, time 1780.0411336421967
Evaluate loss 3.3437745571136475, time 1780.4640264511108
Evaluate loss 3.4028284549713135, time 1780.8840250968933
Evaluate loss 3.3693811893463135, time 1781.3115940093994
Evaluate loss 3.331863403320

Evaluate loss 3.0347626209259033, time 1835.5424077510834
Evaluate loss 3.0372257232666016, time 1835.96093416214
Evaluate loss 2.9577596187591553, time 1836.390502691269
Evaluate loss 2.883840799331665, time 1836.8114359378815
Evaluate loss 2.8818352222442627, time 1837.2328181266785
Evaluate loss 2.8982694149017334, time 1837.6582870483398
Evaluate loss 2.8827755451202393, time 1838.081295967102
Evaluate loss 2.945297956466675, time 1838.5071878433228
Evaluate loss 3.0445239543914795, time 1838.9285695552826
Evaluate loss 3.0603668689727783, time 1839.3507301807404
Evaluate loss 3.0163791179656982, time 1839.7738497257233
Evaluate loss 2.941213369369507, time 1840.1981575489044
Evaluate loss 2.9534575939178467, time 1840.6266694068909
Evaluate loss 2.9261269569396973, time 1841.0471913814545
Evaluate loss 2.9878015518188477, time 1841.472663640976
Evaluate loss 2.9918060302734375, time 1841.8937335014343
Evaluate loss 3.0835094451904297, time 1842.3149893283844
Evaluate loss 3.173451

In [12]:
with open('wiki_bak/test_char.npy', 'rb') as inp:
    test_char = np.load(inp)
with open('wiki_bak/test_word.npy', 'rb') as inp:
    test_word = np.load(inp)
print(len(test_word), len(test_char))

12086755 12086755


In [14]:
test_word = batchify(test_word, 47).T
test_char = batchify(test_char, 47).T

In [ ]:
evaluate_step(test_word, test_char, 70)

Evaluate loss 3.426823616027832, time 0.6646745204925537
Evaluate loss 3.1074776649475098, time 1.1751036643981934
Evaluate loss 3.18856143951416, time 1.6819932460784912
Evaluate loss 3.317288398742676, time 2.262643337249756
Evaluate loss 3.3311283588409424, time 2.7695250511169434
Evaluate loss 3.273428201675415, time 3.3198299407958984
Evaluate loss 3.262254238128662, time 3.9047272205352783
Evaluate loss 3.2594146728515625, time 4.4148242473602295
Evaluate loss 3.0967628955841064, time 4.956952095031738
Evaluate loss 3.2132723331451416, time 5.536021709442139
Evaluate loss 3.1424825191497803, time 6.046091318130493
Evaluate loss 3.2476983070373535, time 6.556710243225098
Evaluate loss 3.197664499282837, time 7.065145492553711
Evaluate loss 3.214688539505005, time 7.570648431777954
Evaluate loss 3.1736090183258057, time 8.083530187606812
Evaluate loss 3.247009754180908, time 8.59142541885376
Evaluate loss 3.3124442100524902, time 9.100245237350464
Evaluate loss 3.298973560333252, t

Evaluate loss 3.254096746444702, time 75.34829211235046
Evaluate loss 3.254969596862793, time 75.85474014282227
Evaluate loss 3.261353015899658, time 76.36489915847778
Evaluate loss 3.2553725242614746, time 76.8739800453186
Evaluate loss 3.178994655609131, time 77.38697075843811
Evaluate loss 3.1756138801574707, time 77.89813709259033
Evaluate loss 3.062457323074341, time 78.40631628036499
Evaluate loss 3.074418306350708, time 78.9185996055603
Evaluate loss 3.158013105392456, time 79.43700051307678
Evaluate loss 3.0106561183929443, time 79.94947123527527
Evaluate loss 3.1168737411499023, time 80.46432852745056
Evaluate loss 3.215427875518799, time 80.98098373413086
Evaluate loss 3.1176202297210693, time 81.49524426460266
Evaluate loss 3.1794261932373047, time 82.00641465187073
Evaluate loss 3.2265422344207764, time 82.5178050994873
Evaluate loss 3.20131254196167, time 83.03356266021729
Evaluate loss 3.0893046855926514, time 83.54402017593384
Evaluate loss 3.1787946224212646, time 84.05

Evaluate loss 3.3341615200042725, time 255.03346467018127
Evaluate loss 3.2856459617614746, time 255.54936981201172
Evaluate loss 3.2871999740600586, time 256.0586402416229
Evaluate loss 3.3121118545532227, time 256.57190322875977
Evaluate loss 3.144472360610962, time 257.08709836006165
Evaluate loss 3.208768844604492, time 257.6056582927704
Evaluate loss 3.2917826175689697, time 258.118763923645
Evaluate loss 3.3137805461883545, time 258.62952280044556
Evaluate loss 3.325410842895508, time 259.14283537864685
Evaluate loss 3.272937059402466, time 259.6573255062103
Evaluate loss 3.1950912475585938, time 260.1716854572296
Evaluate loss 3.236769914627075, time 260.682030916214
Evaluate loss 3.201514959335327, time 261.1943008899689
Evaluate loss 3.175776481628418, time 261.70722794532776
Evaluate loss 3.179445266723633, time 262.2239317893982
Evaluate loss 3.208111047744751, time 262.7337408065796
Evaluate loss 3.1847128868103027, time 263.244686126709
Evaluate loss 3.239481210708618, tim

Evaluate loss 3.204246759414673, time 329.55932235717773
Evaluate loss 3.2037365436553955, time 330.07233595848083
Evaluate loss 3.011934757232666, time 330.58454060554504
Evaluate loss 3.028376817703247, time 331.097279548645
Evaluate loss 3.171438694000244, time 331.60947608947754
Evaluate loss 3.1298418045043945, time 332.11787128448486
Evaluate loss 3.203814744949341, time 332.6294331550598
Evaluate loss 3.0403525829315186, time 333.1405384540558
Evaluate loss 3.0676214694976807, time 333.65314507484436
Evaluate loss 3.086202383041382, time 334.16392970085144
Evaluate loss 3.118312358856201, time 334.68044424057007
Evaluate loss 2.9681570529937744, time 335.19389486312866
Evaluate loss 3.0369787216186523, time 335.7166714668274
Evaluate loss 2.991346836090088, time 336.23479557037354
Evaluate loss 3.0630075931549072, time 336.7520763874054
Evaluate loss 3.062129497528076, time 337.26657366752625
Evaluate loss 3.0946524143218994, time 337.78107714653015
Evaluate loss 3.1346237659454

Evaluate loss 3.0426042079925537, time 403.6311287879944
Evaluate loss 3.048707962036133, time 404.1434783935547
Evaluate loss 3.194470167160034, time 404.6614043712616
Evaluate loss 3.0601158142089844, time 405.17529797554016
Evaluate loss 3.0165867805480957, time 405.6905882358551
Evaluate loss 3.124058485031128, time 406.2024025917053
Evaluate loss 3.225473165512085, time 406.72028493881226
Evaluate loss 3.189363479614258, time 407.2371916770935
Evaluate loss 3.1936323642730713, time 407.7547516822815
Evaluate loss 3.185748815536499, time 408.26978969573975
Evaluate loss 3.0864343643188477, time 408.7839705944061
Evaluate loss 2.9705774784088135, time 409.2992627620697
Evaluate loss 3.060917377471924, time 409.81096959114075
Evaluate loss 3.015009641647339, time 410.32243609428406
Evaluate loss 3.0325403213500977, time 410.8371193408966
Evaluate loss 2.9758238792419434, time 411.3500335216522
Evaluate loss 2.944650173187256, time 411.8618404865265
Evaluate loss 2.9623324871063232, t

...

Evaluate loss 3.1132421493530273, time 702.6508042812347
Evaluate loss 3.247130870819092, time 703.1628999710083
Evaluate loss 3.2646071910858154, time 703.6732850074768
Evaluate loss 3.2849111557006836, time 704.1877136230469
Evaluate loss 3.24334454536438, time 704.7024612426758
Evaluate loss 3.225931167602539, time 705.2264041900635
Evaluate loss 3.222224712371826, time 705.7447466850281
Evaluate loss 3.190579652786255, time 706.2640342712402
Evaluate loss 3.2376110553741455, time 706.7829008102417
Evaluate loss 3.1648550033569336, time 707.2980921268463
Evaluate loss 3.2293519973754883, time 707.8108973503113
Evaluate loss 3.223614454269409, time 708.3264646530151
Evaluate loss 3.1575820446014404, time 708.8401529788971
Evaluate loss 3.2034873962402344, time 709.3541865348816
Evaluate loss 3.2306020259857178, time 709.8688495159149
Evaluate loss 3.142831563949585, time 710.379575252533
Evaluate loss 3.127593755722046, time 710.8930821418762
Evaluate loss 3.3360021114349365, time 71

Evaluate loss 3.3424623012542725, time 777.8626215457916
Evaluate loss 3.3286807537078857, time 778.3712048530579
Evaluate loss 3.270808696746826, time 778.8863079547882
Evaluate loss 3.28529691696167, time 779.4004671573639
Evaluate loss 3.343400239944458, time 779.9142489433289
Evaluate loss 3.352647304534912, time 780.4289977550507
Evaluate loss 3.2766878604888916, time 780.94224858284
Evaluate loss 3.317972421646118, time 781.4558098316193
Evaluate loss 3.2495832443237305, time 781.9686722755432
Evaluate loss 3.317946672439575, time 782.4852242469788
Evaluate loss 3.235711097717285, time 783.0012831687927
Evaluate loss 3.249521493911743, time 783.5181496143341
Evaluate loss 3.2801239490509033, time 784.0336673259735
Evaluate loss 3.2052161693573, time 784.5495564937592
Evaluate loss 3.168164014816284, time 785.0637953281403
Evaluate loss 3.1681456565856934, time 785.5830795764923
Evaluate loss 3.18886399269104, time 786.1003563404083
Evaluate loss 3.155500888824463, time 786.618875

Evaluate loss 3.103529214859009, time 853.0554802417755
Evaluate loss 3.140167236328125, time 853.5685658454895
Evaluate loss 3.1706395149230957, time 854.0803823471069
Evaluate loss 3.322493076324463, time 854.5922493934631
Evaluate loss 3.174830198287964, time 855.1014361381531
Evaluate loss 3.0683302879333496, time 855.6145629882812
Evaluate loss 2.997838020324707, time 856.1282765865326
Evaluate loss 3.0164036750793457, time 856.641030550003
Evaluate loss 3.0434815883636475, time 857.1528396606445
Evaluate loss 3.1474392414093018, time 857.6722629070282
Evaluate loss 3.0628721714019775, time 858.1911396980286
Evaluate loss 3.1721012592315674, time 858.7086853981018
Evaluate loss 3.2069003582000732, time 859.2247533798218
Evaluate loss 3.196504592895508, time 859.7434501647949
Evaluate loss 3.2112293243408203, time 860.258419752121
Evaluate loss 3.2024292945861816, time 860.7725455760956
Evaluate loss 3.1675662994384766, time 861.2877032756805
Evaluate loss 3.137932062149048, time 8

Evaluate loss 3.257352352142334, time 928.2213492393494
Evaluate loss 3.1683759689331055, time 928.7355079650879
Evaluate loss 3.252654790878296, time 929.2467682361603
Evaluate loss 3.22224760055542, time 929.7594883441925
Evaluate loss 3.102409839630127, time 930.2720437049866
Evaluate loss 3.164146900177002, time 930.8027827739716
Evaluate loss 3.11171293258667, time 931.3186941146851
Evaluate loss 3.2166295051574707, time 931.8293058872223
Evaluate loss 3.1524171829223633, time 932.3422820568085
Evaluate loss 3.2246313095092773, time 932.8574194908142
Evaluate loss 3.2018604278564453, time 933.3792457580566
Evaluate loss 3.3308095932006836, time 933.896454334259
Evaluate loss 3.2899441719055176, time 934.4188091754913
Evaluate loss 3.3175716400146484, time 934.9372284412384
Evaluate loss 3.232320547103882, time 935.4522724151611
Evaluate loss 3.133009910583496, time 935.9656364917755
Evaluate loss 2.994690418243408, time 936.4787886142731
Evaluate loss 3.0837204456329346, time 936.

Evaluate loss 3.248990297317505, time 1003.4692511558533
Evaluate loss 3.162106990814209, time 1003.9816949367523
Evaluate loss 3.090632677078247, time 1004.5001347064972
Evaluate loss 3.163100242614746, time 1005.0107691287994
Evaluate loss 3.1779558658599854, time 1005.5197281837463
Evaluate loss 3.153005599975586, time 1006.0307152271271
Evaluate loss 3.1900548934936523, time 1006.5426845550537
Evaluate loss 3.2215147018432617, time 1007.0629336833954
Evaluate loss 3.0338916778564453, time 1007.5822780132294
Evaluate loss 3.0298712253570557, time 1008.0974214076996
Evaluate loss 3.1330173015594482, time 1008.6160840988159
Evaluate loss 3.011289596557617, time 1009.131445646286
Evaluate loss 3.006398916244507, time 1009.6458601951599
Evaluate loss 3.202924966812134, time 1010.1595118045807
Evaluate loss 3.136786699295044, time 1010.6802818775177
Evaluate loss 3.1900835037231445, time 1011.1922919750214
Evaluate loss 3.270860433578491, time 1011.7049844264984
Evaluate loss 3.272119998

In [ ]:
d11